In [38]:
EG = "/Users/joregan/Playing/merged_annotations/hsi_4_0717_209_002.json"

In [39]:
import json

In [40]:
with open(EG) as inf:
    data = json.load(inf)

In [9]:
SAMPLE = """
{
  "2": {
    "general": {
      "id": 2,
      "start": 5.970287890159987,
      "end": 9.551266621561616
    },
    "high_level": {
      "topic_name": "reference_real",
      "current_topic": "vase",
      "topic_change": false,
      "topic_duration_id": 0,
      "spatial_reference": "fourth vase",
      "referenced_object": [
        "vase"
      ]
    },
    "low_level": {
      "resolved_references": {
        "the fourth vase": "vase",
        "it": "vase"
      },
      "spatial_relationships": [],
      "resolved_adverbs": []
    },
    "snippet": "But I'm wondering about the fourth vase. Where is it? Did you break it?"
  }
}
"""

In [10]:
thing = json.loads(SAMPLE.strip())

In [13]:
thing["2"].keys()

dict_keys(['general', 'high_level', 'low_level', 'snippet'])

In [14]:
thing["2"]["low_level"]

{'resolved_references': {'the fourth vase': 'vase', 'it': 'vase'},
 'spatial_relationships': [],
 'resolved_adverbs': []}

In [16]:
thing["2"]["low_level"]["resolved_references"] == {}

False

In [18]:
words = [x.lower().strip(".;?") for x in thing["2"]["snippet"].split(" ")]

In [ ]:
for key in data:
    if data[key]["low_level"]["resolved_references"] == {}:
        print(f"{key}\t{data[key]['snippet']}")

In [ ]:
unreal = ["4"]
additions_real = {
    "4": {

    }
}
fixes = {
    "5": {
        "real": {
            "See-through": "vase",
            "blue": "vase",
            "green": "vase"
        },
        "unreal": {
            "black": "vase"
        }
    }
}

In [23]:
STRING = "It's like, it looks weird. Did you do something with it?"

def get_simple_indices(text, check=["it", "it's"]):
    ret = []
    words = [x.lower().strip(",?.;:") for x in text.split(" ")]
    for w in enumerate(words):
        if w[1] in check:
            ret.append(w[0])
    return ret

In [37]:
get_simple_indices("And where did i buy it? I'm I'm not sure... I think it was here.")

[5, 12]

```json
      "resolved_references_indices": {
        "it": [2, 4]
      }
```

In [42]:
data["12"]

{'general': {'id': 12, 'start': 53.02430161961156, 'end': 57.88831036189313},
 'high_level': {'topic_name': 'reference_real',
  'current_topic': 'desk',
  'topic_change': False,
  'topic_duration_id': 0,
  'spatial_reference': '',
  'referenced_object': ['desk']},
 'low_level': {'resolved_references': {'the desk': 'desk', 'it': 'desk'},
  'spatial_relationships': [],
  'resolved_adverbs': [],
  'resolved_references_indices': {'it': [11, 15]}},
 'snippet': 'And how about the cleaning is like the, the desk? Is it, did you dust it?'}

In [ ]:
def load_tsv(filename):
    data = []
    with open(filename) as inf:
        for line in inf.readlines():
            parts = line.strip().split("\t")
            data.append({
                "start": float(parts[0]),
                "end": float(parts[1]),
                "word": parts[2]
            })
    return data

In [47]:
def slice_tsv_data(data, start, end):
    ret = []
    for datum in data:
        if type(datum["start"]) is str:
            datum["start"] = float(datum["start"])
        if type(datum["end"]) is str:
            datum["end"] = float(datum["end"])
        if datum["start"] >= start and datum["end"] <= end:
            ret.append(datum)
        elif datum["end"] > end:
            return ret
    return ret

In [50]:
def clean_text(text):
    return " ".join([x.lower().strip(".,;?!") for x in text.split(" ")])

In [51]:
from pathlib import Path

TSVS = Path("/Users/joregan/Playing/hsi/word_annotations/")
JSON = Path("/Users/joregan/Playing/merged_annotations/")

In [53]:
data["12"]["snippet"]

'And how about the cleaning is like the, the desk? Is it, did you dust it?'

In [58]:
def process_segment(segment, tsv_data):
    start = segment["general"]["start"]
    end = segment["general"]["end"]

    tsv = slice_tsv_data(tsv_data, start, end)
    tsv_words = " ".join([x["word"] for x in tsv])

    if segment["snippet"] != tsv_words:
        print("🙀 mismatch:", segment["snippet"], tsv_words)

In [56]:
sliced = slice_tsv_data(ttsv, data["12"]["general"]["start"], data["12"]["general"]["end"])

In [57]:
data["12"]["snippet"] == " ".join([x["word"] for x in sliced])

True